In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, avg, corr
from pyspark.sql.types import DoubleType, IntegerType, StringType, BooleanType, DateType, StructType, StructField
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Initialize SparkSession with memory optimization
spark = SparkSession.builder \
    .appName("Spotify Data Cleaning") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/05 18:51:19 INFO SparkEnv: Registering MapOutputTracker
25/04/05 18:51:19 INFO SparkEnv: Registering BlockManagerMaster
25/04/05 18:51:19 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/04/05 18:51:19 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
# Load data from GCP DataProc
file_path = "gs://dataproc-staging-us-central1-361128386781-eo9ksqfa/merged_data.csv"

# Define explicit schema
schema = StructType([
    StructField("id", StringType(), True),  # String ID like "0", "1", etc.
    StructField("title", StringType(), True),  # Track title
    StructField("rank", IntegerType(), True),  # Numeric rank on chart
    StructField("date", DateType(), True),  # Date format like "2017-01-01"
    StructField("artist", StringType(), True),  # Artist name(s)
    StructField("url", StringType(), True),  # Spotify URL
    StructField("region", StringType(), True),  # Region like "Argentina"
    StructField("chart", StringType(), True),  # Chart type like "top200"
    StructField("trend", StringType(), True),  # Trend like "SAME_POSITION", "MOVE_UP"
    StructField("streams", DoubleType(), True),  # Number of streams (appears as float in sample)
    StructField("track_id", StringType(), True),  # Unique track identifier
    StructField("album", StringType(), True),  # Album title
    StructField("popularity", DoubleType(), True),  # Popularity score (float in sample)
    StructField("duration_ms", DoubleType(), True),  # Duration in ms (float in sample)
    StructField("explicit", BooleanType(), True),  # Boolean as "True" or "False" strings
    StructField("release_date", StringType(), True),  # Release date as string
    StructField("available_markets", StringType(), True),  # String representing array of markets
    StructField("af_danceability", DoubleType(), True),  # Audio feature: danceability
    StructField("af_energy", DoubleType(), True),  # Audio feature: energy
    StructField("af_key", DoubleType(), True),  # Audio feature: key
    StructField("af_loudness", DoubleType(), True),  # Audio feature: loudness
    StructField("af_mode", DoubleType(), True),  # Audio feature: mode
    StructField("af_speechiness", DoubleType(), True),  # Audio feature: speechiness
    StructField("af_acousticness", DoubleType(), True),  # Audio feature: acousticness
    StructField("af_instrumentalness", DoubleType(), True),  # Audio feature: instrumentalness
    StructField("af_liveness", DoubleType(), True),  # Audio feature: liveness
    StructField("af_valence", DoubleType(), True),  # Audio feature: valence
    StructField("af_tempo", DoubleType(), True),  # Audio feature: tempo
    StructField("af_time_signature", DoubleType(), True)  # Audio feature: time signature
])

# Load the data with explicit schema instead of inferring
spotify_df = spark.read.csv(file_path, header=True, schema=schema)

# Print data schema
print("Data Schema:")
spotify_df.printSchema()

Data Schema:
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: double (nullable = true)
 |-- track_id: string (nullable = true)
 |-- album: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- release_date: string (nullable = true)
 |-- available_markets: string (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumenta

In [4]:
# Count number of unique songs by track_id
unique_tracks = spotify_df.select("track_id").distinct().count()
print(f"Number of unique tracks: {unique_tracks}")

Number of unique tracks: 200290


In [5]:
# Drop specified columns
columns_to_drop = ["id", "rank", "date", "url", "chart", "trend", "streams", "release_date", "available_markets"]
spotify_df_reduced = spotify_df.drop(*columns_to_drop)

# Show remaining columns
print("Remaining columns after dropping specified ones:")
spotify_df_reduced.printSchema()

Remaining columns after dropping specified ones:
root
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- region: string (nullable = true)
 |-- track_id: string (nullable = true)
 |-- album: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- af_danceability: double (nullable = true)
 |-- af_energy: double (nullable = true)
 |-- af_key: double (nullable = true)
 |-- af_loudness: double (nullable = true)
 |-- af_mode: double (nullable = true)
 |-- af_speechiness: double (nullable = true)
 |-- af_acousticness: double (nullable = true)
 |-- af_instrumentalness: double (nullable = true)
 |-- af_liveness: double (nullable = true)
 |-- af_valence: double (nullable = true)
 |-- af_tempo: double (nullable = true)
 |-- af_time_signature: double (nullable = true)



In [6]:
# Count rows before deduplication
before_dedup = spotify_df_reduced.count()
print(f"Row count before deduplication: {before_dedup}")

# Deduplicate based on track_id
spotify_df_deduplicated = spotify_df_reduced.dropDuplicates(["track_id"])

# Count rows after deduplication
after_dedup = spotify_df_deduplicated.count()
print(f"Row count after deduplication: {after_dedup}")
print(f"Removed {before_dedup - after_dedup} duplicate rows ({(before_dedup - after_dedup) / before_dedup * 100:.2f}% of data)")

Row count before deduplication: 26174269


Row count after deduplication: 200290
Removed 25973979 duplicate rows (99.23% of data)


In [7]:
# Step 1: Save the deduplicated data to CSV
output_path = "gs://dataproc-staging-us-central1-361128386781-eo9ksqfa/spotify_deduplicated_data.csv"
print(f"Saving deduplicated data to: {output_path}")
spotify_df_deduplicated.write.mode("overwrite").option("header", "true").csv(output_path)
print("Data saved successfully!")

Saving deduplicated data to: gs://dataproc-staging-us-central1-361128386781-eo9ksqfa/spotify_deduplicated_data.csv


25/04/05 19:01:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Data saved successfully!
